<a href="https://colab.research.google.com/github/LukegCashman/Cashman_ECGR4105/blob/main/LC_HW7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# SECTION 0: Initialize Data Sets

import torch
import torch.optim as optim
import torch.nn.functional as F
import collections
import numpy as np
import pandas as pd
import torch.nn as nn
from collections import OrderedDict

In [ ]:
# Label class names
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [ ]:
# Pull test and validation sets for Cifar 10
from torchvision import datasets, transforms
data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True,
                           transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))]))
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True,
                           transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))]))

100%|██████████| 170M/170M [00:05<00:00, 30.8MB/s]


Extracting ../data-unversioned/p1ch7/cifar-10-python.tar.gz to ../data-unversioned/p1ch7/
Files already downloaded and verified


In [ ]:
# SECTION 1: Problem 1.a

conv = nn.Conv2d(3, 16, kernel_size=3) # <1>
conv

Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))

In [ ]:
img, _ = cifar10[0]
output = conv(img.unsqueeze(0))
img.unsqueeze(0).shape, output.shape

(torch.Size([1, 3, 32, 32]), torch.Size([1, 16, 30, 30]))

In [ ]:
with torch.no_grad():
    conv.bias.zero_()

with torch.no_grad():
    conv.weight.fill_(1.0 / 9.0)

In [ ]:
pool = nn.MaxPool2d(2)
output = pool(img.unsqueeze(0))

img.unsqueeze(0).shape, output.shape

(torch.Size([1, 3, 32, 32]), torch.Size([1, 3, 16, 16]))

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * 8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [ ]:
model = Net()
model(img.unsqueeze(0))

tensor([[ 0.0137, -0.0115, -0.0480,  0.2045,  0.1348,  0.2175,  0.0691,  0.0140,
          0.0098,  0.1647]], grad_fn=<AddmmBackward0>)

In [ ]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)

            outputs = model(imgs)

            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()

            loss.backward()

            optimizer.step()

            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)

model = Net()
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 200,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

2024-12-05 19:47:58.862627 Epoch 1, Training loss 2.0130842849421686
2024-12-05 19:52:02.272715 Epoch 10, Training loss 1.185724200723726
2024-12-05 19:56:29.166826 Epoch 20, Training loss 1.0294857504575148
2024-12-05 20:00:53.291555 Epoch 30, Training loss 0.9442758584571311
2024-12-05 20:05:13.456982 Epoch 40, Training loss 0.880814834095328
2024-12-05 20:09:37.953248 Epoch 50, Training loss 0.8328574570396062
2024-12-05 20:13:58.114924 Epoch 60, Training loss 0.7948647441766451
2024-12-05 20:18:18.141039 Epoch 70, Training loss 0.7651457932141735
2024-12-05 20:22:39.133741 Epoch 80, Training loss 0.7409358746407891
2024-12-05 20:27:00.658937 Epoch 90, Training loss 0.7190939601501236
2024-12-05 20:31:18.128537 Epoch 100, Training loss 0.7008688967398671
2024-12-05 20:35:38.560770 Epoch 110, Training loss 0.6807030332286644
2024-12-05 20:40:02.010079 Epoch 120, Training loss 0.6666339024177292
2024-12-05 20:44:29.077716 Epoch 130, Training loss 0.6529947539881977
2024-12-05 20:48:56

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0]  # <3>
                correct += int((predicted == labels).sum())

        print("Accuracy {}: {:.2f}".format(name , correct / total))

validate(model, train_loader, val_loader)

Accuracy train: 0.80
Accuracy val: 0.64


In [ ]:
# SECTION 2: Problem 2.a

conv = nn.Conv2d(3, 16, kernel_size=3) # <1>
conv

Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))

In [ ]:
img, _ = cifar10[0]
output = conv(img.unsqueeze(0))
img.unsqueeze(0).shape, output.shape

(torch.Size([1, 3, 32, 32]), torch.Size([1, 16, 30, 30]))

In [ ]:
with torch.no_grad():
    conv.bias.zero_()

with torch.no_grad():
    conv.weight.fill_(1.0 / 9.0)

In [ ]:
pool = nn.MaxPool2d(2)
output = pool(img.unsqueeze(0))

img.unsqueeze(0).shape, output.shape

(torch.Size([1, 3, 32, 32]), torch.Size([1, 3, 16, 16]))

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(8, 4, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(4 * 4 * 4, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = F.max_pool2d(torch.tanh(self.conv3(out)), 2)
        out = out.view(out.size(0), -1)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [ ]:
train_loader1 = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)

model1 = Net()
optimizer1 = optim.SGD(model.parameters(), lr=1e-2)
loss_fn1 = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 200,
    optimizer = optimizer1,
    model = model1,
    loss_fn = loss_fn1,
    train_loader = train_loader1,
)

2024-12-05 21:17:44.510419 Epoch 1, Training loss 2.302175478557187
2024-12-05 21:21:45.226120 Epoch 10, Training loss 2.302200661900708
2024-12-05 21:26:12.493454 Epoch 20, Training loss 2.3022086830700146
2024-12-05 21:30:39.732848 Epoch 30, Training loss 2.3021884970652784
2024-12-05 21:35:09.898546 Epoch 40, Training loss 2.3021671131748676
2024-12-05 21:39:40.240362 Epoch 50, Training loss 2.3021834658844695
2024-12-05 21:44:16.275163 Epoch 60, Training loss 2.302187675100458
2024-12-05 21:48:56.558062 Epoch 70, Training loss 2.3022021573522817
2024-12-05 21:53:37.276249 Epoch 80, Training loss 2.3021843811435163
2024-12-05 21:58:20.673373 Epoch 90, Training loss 2.3021830216698023
2024-12-05 22:03:02.629735 Epoch 100, Training loss 2.3021985261946383
2024-12-05 22:07:39.653723 Epoch 110, Training loss 2.3022140261462276
2024-12-05 22:12:14.752497 Epoch 120, Training loss 2.3021973624558707
2024-12-05 22:16:48.749083 Epoch 130, Training loss 2.302218446036434
2024-12-05 22:21:22.4

In [ ]:
train_loader1 = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)
val_loader1 = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0]  # <3>
                correct += int((predicted == labels).sum())

        print("Accuracy {}: {:.2f}".format(name , correct / total))

validate(model1, train_loader1, val_loader1)

Accuracy train: 0.11
Accuracy val: 0.11


In [ ]:
# SECTION 3: Problem 2

class ResBlock(nn.Module):
    def __init__(self, n_chans):
        super(ResBlock, self).__init__()
        self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3,
                              padding=1, bias=False)  # <1>
        self.batch_norm = nn.BatchNorm2d(num_features=n_chans)
        torch.nn.init.kaiming_normal_(self.conv.weight,
                                      nonlinearity='relu')  # <2>
        torch.nn.init.constant_(self.batch_norm.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm.bias)

    def forward(self, x):
        out = self.conv(x)
        out = self.batch_norm(out)
        out = torch.relu(out)
        return out + x

In [ ]:
class ResNet10(nn.Module):
    def __init__(self, n_chans1=32, n_blocks=10):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.resblocks = nn.Sequential(
            *(n_blocks * [ResBlock(n_chans=n_chans1)]))
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = self.resblocks(out)
        out = F.max_pool2d(out, 2)
        out = out.view(-1, 8 * 8 * self.n_chans1)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [ ]:
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


In [ ]:
all_acc_dict = collections.OrderedDict()

model = ResNet10(n_chans1=32, n_blocks=100).to(device=device)
optimizer = optim.SGD(model.parameters(), lr=3e-3)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)
all_acc_dict["res deep"] = validate(model, train_loader, val_loader)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
